In [1]:
import os
from sqlalchemy import create_engine
from sqlalchemy import inspect, text
import pandas as pd

db_type = 'postgresql'
db_api = 'psycopg2'
postgres_host = os.getenv('POSTGRES_HOST')
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')
postgres_dbname = os.getenv('POSTGRES_DB')
postgres_port = 5432

print(postgres_host)
print(postgres_dbname)

192.168.1.51
pagila


In [2]:
engine = create_engine(
    f'{db_type}+{db_api}://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_dbname}'
)

In [3]:
engine.execution_options(isolation_level='AUTOCOMMIT').connect()

In [23]:
inspector = inspect(engine)
inspector.get_columns('samuel_movies')

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Title',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Year',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [31]:
samuel_df = pd.read_csv('samuel_movies.csv')
tim_df = pd.read_csv('tim_movies.csv')

In [32]:
print(samuel_df.dtypes)
print(tim_df.dtypes)

Title    object
Year      int64
dtype: object
Title    object
Year      int64
dtype: object


In [33]:
samuel_df.to_sql('samuel_movies', engine, if_exists='replace')
tim_df.to_sql('tim_movies', engine, if_exists='replace')

91

In [36]:
# Movies where both Samuel and Tim had a role.
sql_query_both = text('''SELECT sam."Title", tim."Title" from samuel_movies as sam, 
                      tim_movies as tim WHERE sam."Title" = tim."Title"''')

with engine.connect() as conn:    
    data = conn.execute(sql_query_both)
    print(data)

ProgrammingError: (psycopg2.errors.SyntaxError) unterminated quoted identifier at or near ""Title"
LINE 2: WHERE sam."Title" = tim."Title
                                ^

[SQL: SELECT sam."Title", tim."Title" from samuel_movies as sam, tim_movies as tim
WHERE sam."Title" = tim."Title]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [20]:
movies_both = samuel_df[samuel_df['Title'].eq(tim_df['Title'])
print(movies_both)

442